# Dataset Creation From CSV

In [1]:
!python -m uni2ts.data.builder.simple ETTh1 ./csvs/ETTh1.csv --dataset_type wide --date_offset '2017-10-23 23:00:00'

Saving the dataset (1/1 shards): 100%|████| 7/7 [00:00<00:00, 325.38 examples/s]


# Edit the Config Files

In [2]:
import yaml


config_path = './conf/finetune/default.yaml'

model_name = 'moirai_1.0_R_small'
data_name = 'etth1'
val_data_name = 'etth1'
run_name = 'test-01'

with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

# config['model']['name'] = model_name

config['defaults'][0]['model'] = model_name
config['defaults'][1]['data'] = data_name
config['run_name'] = run_name

with open(config_path, 'w') as f:
    yaml.dump(config, f)

# Hyperparameter Setting

In [3]:
import yaml

model_type = 'small'
model_cfg_path = f'./conf/finetune/model/moirai_1.0_R_{model_type}.yaml'

with open(model_cfg_path, 'r') as f:
    model_cfg = yaml.load(f, Loader=yaml.FullLoader)

# loss = 'uni2ts.loss.packed.PackedNRMSELoss'

# model_cfg['loss_func']['_target_'] = loss
# model_cfg['val_metric'][0]['_target_'] = loss
# model_cfg['val_metric'][1]['_target_'] = loss

# with open(model_cfg_path, 'w') as f:
#     yaml.dump(model_cfg, f)

# Finetuning

In [4]:
!python train.py

cfg model: {'_target_': 'uni2ts.model.moirai.MoiraiFinetune.load_from_checkpoint', 'module_kwargs': {'_target_': 'builtins.dict', 'distr_output': {'_target_': 'uni2ts.distribution.MixtureOutput', 'components': [{'_target_': 'uni2ts.distribution.StudentTOutput'}, {'_target_': 'uni2ts.distribution.NormalFixedScaleOutput'}, {'_target_': 'uni2ts.distribution.NegativeBinomialOutput'}, {'_target_': 'uni2ts.distribution.LogNormalOutput'}]}, 'd_model': 384, 'num_layers': 6, 'patch_sizes': '${as_tuple:[8, 16, 32, 64, 128]}', 'max_seq_len': 512, 'attn_dropout_p': 0.0, 'dropout_p': 0.0, 'scaling': True}, 'min_patches': 2, 'min_mask_ratio': 0.15, 'max_mask_ratio': 0.5, 'max_dim': 128, 'loss_func': {'_target_': 'uni2ts.loss.packed.PackedNLLLoss'}, 'val_metric': [{'_target_': 'uni2ts.loss.packed.PackedMSELoss'}, {'_target_': 'uni2ts.loss.packed.PackedNRMSELoss', 'normalize': 'absolute_target_squared'}], 'lr': 0.001, 'weight_decay': 0.1, 'beta1': 0.9, 'beta2': 0.98, 'num_training_steps': '${mul:${tra

In [5]:
# from hydra.utils import instantiate
# import hydra
# from lora import lora_finetune
# import pytorch_lightning as L

# @hydra.main(version_base="1.3", config_path='./conf/finetune', config_name="default.yaml")
# def load_model(cfg):
#     model: L.LightningModule = instantiate(cfg.model, _convert_="all")
#     model = lora_finetune(model).get_base_model()
#     return model


In [6]:
# %tb
# model = load_model()